In [21]:
from bw_processing.constants import DEFAULT_LICENSES
from bw_processing.filesystem import clean_datapackage_name, safe_filename
from bw_processing.utils import check_name
from pathlib import Path
from typing import Optional
import datetime
import json
import pandas as pd
import uuid

Had some trouble here as I didn't realize that `indicators` originally comes with the same length as `cfs`, so the joins were in effect cross products. One could in theory join these two `DataFrames` based on the indices, but I feel more comfortable using `drop_duplicates` and actually joining on the data attributes.

In [2]:
def load_formatted_dataframe(excel_fp, separator="|"):
    cfs = pd.read_excel(
        excel_fp, 
        sheet_name="CFs"
    ).rename(columns={'CF': 'Characterization factor', 'Name': 'Flowable'})
    indicators = pd.read_excel(
        "/Users/chrismutel/Sync/Documents/LCA/Ecoinvent/3.9/LCIA/LCIA Implementation 3.9.xlsx", 
        sheet_name="Indicators"
    ).drop_duplicates().rename(columns={'Unit': 'Indicator unit'})
    cfs_merged = pd.merge(cfs, indicators, how='left', on=['Method', 'Category', 'Indicator'])
    assert len(cfs_merged) == len(cfs)
    
    cfs_merged['Context'] = cfs.apply(lambda row: row['Compartment'] + separator + row['Subcompartment'], axis=1)
    cfs_merged['Indicator'] = cfs.apply(lambda row: row['Category'] + separator + row['Indicator'], axis=1)
    cfs_merged.drop(columns=['Compartment', 'Subcompartment', 'Category'], inplace=True)
    return cfs_merged

In [3]:
df = load_formatted_dataframe(
    "/Users/chrismutel/Sync/Documents/LCA/Ecoinvent/3.9/LCIA/LCIA Implementation 3.9.xlsx", 
)

In [4]:
METHOD_METADATA = json.load(open("methods.json"))

In [5]:
INDICATOR_MAPPING = {
    (row['method'], row['indicator']): row['uuid'] 
    for row in json.load(open("indicators.json"))
}

In [6]:
flows = pd.read_csv(Path.cwd().parent / "Elementary flow mapping" / "outputs" / "ecoinvent-3.9" / "ecoinvent-3.9.csv")

In [7]:
df['Method UUID'] = df.apply(lambda row: METHOD_METADATA[row['Method']]['uuid'], axis=1)

In [8]:
df['Indicator UUID'] = df.apply(lambda row: INDICATOR_MAPPING[(row['Method'], row['Indicator'])], axis=1)

In [9]:
df = pd.merge(df, flows, how='left', on=['Flowable', 'Context'])

In [10]:
df.drop(columns=['Formula', 'Synonyms', 'Class', 'ExternalReference', 'Preferred', 'AltUnit', 'Second CAS'], inplace=True)

In [12]:
df = df[['Method', 'Method UUID', 'Indicator', 'Indicator UUID', 
         'Indicator unit', 'Flowable', 'Flow UUID', 'Context', 
         'Unit', 'CAS No', 'Characterization factor']]
df

,Method,Method UUID,Indicator,Indicator UUID,Indicator unit,Flowable,Flow UUID,Context,Unit,CAS No,Characterization factor
0,CML v4.8 2016 no LT,b8355520a26f4b1584782411ec5bb467,"acidification no LT|acidification (incl. fate,...",7da02f593a8249a99ccbc19cefe795cc,kg SO2-Eq,Ammonia,9990b51b-7023-4700-bca0-1a32ef921f74,air|urban air close to ground,kg,0007664-41-7,1.600000
1,CML v4.8 2016 no LT,b8355520a26f4b1584782411ec5bb467,"acidification no LT|acidification (incl. fate,...",7da02f593a8249a99ccbc19cefe795cc,kg SO2-Eq,Sulfur dioxide,8c52f40c-69b7-4538-8923-b371523c71f5,air|urban air close to ground,kg,0007446-09-5,1.200000
2,CML v4.8 2016 no LT,b8355520a26f4b1584782411ec5bb467,"acidification no LT|acidification (incl. fate,...",7da02f593a8249a99ccbc19cefe795cc,kg SO2-Eq,Sulfuric acid,7ecc2b66-8dde-4266-8832-f492f564377b,air|urban air close to ground,kg,0007664-93-9,0.783673
3,CML v4.8 2016 no LT,b8355520a26f4b1584782411ec5bb467,"acidification no LT|acidification (incl. fate,...",7da02f593a8249a99ccbc19cefe795cc,kg SO2-Eq,Ammonia,0f440cc0-0f74-446d-99d6-8ff0e97a2444,air|non-urban air or from high stacks,kg,0007664-41-7,1.600000
4,CML v4.8 2016 no LT,b8355520a26f4b1584782411ec5bb467,"acidification no LT|acidification (incl. fate,...",7da02f593a8249a99ccbc19cefe795cc,kg SO2-Eq,Ammonia,8494ed3c-0416-4aa5-b100-51a2b2bcadbd,air|lower stratosphere + upper troposphere,kg,0007664-41-7,1.600000
...,...,...,...,...,...,...,...,...,...,...,...
227218,EN15804,6dfa152a7d5f488c961cefc53449b742,inventory indicators ISO21930|Cumulative Energ...,5ecb89d132fa45dcac3d36507a9a8a4d,MJ,"Gas, natural",7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,natural resource|in ground,Sm3,0008006-14-2,36.000000
227219,EN15804,6dfa152a7d5f488c961cefc53449b742,inventory indicators ISO21930|Cumulative Energ...,5ecb89d132fa45dcac3d36507a9a8a4d,MJ,"Oil, crude",88d06db9-59a1-4719-9174-afeb1fa4026a,natural resource|in ground,kg,NaN,43.400000
227220,EN15804,6dfa152a7d5f488c961cefc53449b742,inventory indicators ISO21930|Cumulative Energ...,5ecb89d132fa45dcac3d36507a9a8a4d,MJ,Peat,c5035ce2-5ee5-431f-a287-4b25da42be74,natural resource|biotic,kg,NaN,9.760000
227221,EN15804,6dfa152a7d5f488c961cefc53449b742,inventory indicators ISO21930|Cumulative Energ...,5ecb89d132fa45dcac3d36507a9a8a4d,MJ,Uranium,2ba5e39b-adb6-4767-a51d-90c1cf32fe98,natural resource|in ground,kg,0007440-61-1,560000.000000


Split into separate dataframes for each method family:

In [13]:
def clean(s):
    return s.replace(" ", "_").replace(",", "")

In [15]:
ALL_METHODS = sorted(df['Method'].unique())

In [16]:
METHOD_RESOURCE_METADATA = []

for method in ALL_METHODS:
    assert method in METHOD_METADATA
    filename = safe_filename(clean(method), add_hash=False)

    # TODO: Could specify a specific resource profile just for LCIA data 
    # to avoid repeating columns, and for data validation
    
    METHOD_RESOURCE_METADATA.append({
        "path": f"{filename}.csv",
        "profile": "tabular-data-resource",
        "mediatype": "text/csv",
        "separator": "|",
        "schema": {
            "fields": [
                {'name': 'Method', 'type': 'string'},
                {'name': 'Method UUID', 'type': 'string'},
                {'name': 'Indicator', 'type': 'string', 'separated': True},
                {'name': 'Indicator UUID', 'type': 'string'},
                {'name': 'Indicator unit', 'type': 'string'},
                {'name': 'Flowable', 'type': 'string'},
                {'name': 'Flow UUID', 'type': 'string'},
                {'name': 'Context', 'type': 'string', 'separated': True},
                {'name': 'Unit', 'type': 'string'},
                {'name': 'CAS No', 'type': 'string'},
                {'name': 'Characterization factor', 'type': 'number'},
            ],
        },
    })

In [18]:
METHOD_RESOURCES = []

for method in ALL_METHODS:
    METHOD_RESOURCES.append(df[df.Method == method])

In [19]:
ECOINVENT_EULA_LICENSE = {
    "name": "ecoinvent-eula-2022.05.01",
    "path": "https://ecoinvent.org/wp-content/uploads/2022/04/ecoinvent_new-db-eula_01_04_2022.pdf",
    "title": "ecoinvent End User Licence Agreement effect 2022.05.01",
}

In [23]:
def to_datapackage(
    dirpath: Path,
    resources: list,
    resources_metadata: list,
    name: str,
    author: str,
    description: str,
    elementary_flow_lists: Optional[list] = None,
    version: Optional[str] = None,
    id_: Optional[str] = None,
    licenses: Optional[list] = None,
):
    dirpath.mkdir(exist_ok=True)

    name = clean_datapackage_name(name)
    check_name(name)

    metadata = {
        "profile": "tabular-data-package",  # https://dataprotocols.org/tabular-data-package/
        "name": name,
        "description": description,
        "id": id_ or uuid.uuid4().hex,
        "licenses": licenses or DEFAULT_LICENSES,
        "resources": resources_metadata,
        "created": datetime.datetime.utcnow().isoformat("T") + "Z",
    }

    json.dump(metadata, open(dirpath / "metadata.json", "w"), indent=2, ensure_ascii=False)
    
    for df, meta in zip(resources, resources_metadata):
        df.to_csv(dirpath / meta['path'], index=False)

In [24]:
to_datapackage(
    dirpath=Path.cwd() / "outputs",
    resources=METHOD_RESOURCES,
    resources_metadata=METHOD_RESOURCE_METADATA,
    name="ecoinvent-3.9-lcia",
    author="Thomas Sonderegger",
    description="Implementation of selected LCIA methods as described in https://v39.ecoquery.ecoinvent.org/File/File?fileName=ecoinvent+3.9+(2022)%2c+current%5csupporting+documents%5cecoinvent+3.9_LCIA_implementation.7z&hash=85940519&type=Files",
    elementary_flow_lists=["ecoinvent-3.9"],
    version="1.0",
    licenses=[ECOINVENT_EULA_LICENSE],
)